In [2]:
!nvidia-smi

Mon Jul  1 07:57:39 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!pip3 install torch torchvision torchaudio

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [4]:
import torch
from torch.utils.data import TensorDataset, DataLoader

torch.manual_seed(1)

train_X = torch.load("./train_X.pt")
train_Y = torch.load("./train_Y.pt")
valid_X = torch.load("./valid_X.pt")
valid_Y = torch.load("./valid_Y.pt")

train_data = TensorDataset(train_X, train_Y)
valid_data = TensorDataset(valid_X, valid_Y)

In [13]:
device = torch.device("cuda")
device

device(type='cuda')

In [15]:
from torch import nn

# nn.Moduleを継承してモデルを作成
class Perseptron(nn.Module):
  def __init__(self, input_size, output_size) -> None:
    super().__init__()

    # nn.Linear: 全結合層を作成する
    self.fc = nn.Linear(input_size, output_size, bias=False)

    # 重み行列を正規分布に上書き
    nn.init.normal_(self.fc.weight, 0.0, 1.0)

  def forward(self, x):
    return torch.softmax(self.fc(x), dim=-1)

model = Perseptron(train_X.size()[1], 4)
model.load_state_dict(torch.load("./knock73.model.pt", map_location=device))
model.to(device)
loss_fn = nn.CrossEntropyLoss()

In [10]:
from torch import optim

op = optim.SGD(model.parameters(), lr=0.1)

In [11]:
def calc(model, data, label):
  size = len(data)
  pred = model(data)
  loss = loss_fn(pred, label)
  accuracy = (pred.argmax(dim = 1) == label).sum().item() / size

  return loss, accuracy

In [24]:
import time

epochs = 10
batches = [1, 2, 4, 8, 16, 32, 64, 128]
losses = torch.zeros((2, epochs)).to(device)
accuracies = torch.zeros(2, epochs).to(device)

for batch in batches:
  train_dataloader = DataLoader(train_data, batch_size=batch, shuffle=True)
  valid_dataloader = DataLoader(valid_data, batch_size=len(valid_data), shuffle=False)

  start = time.perf_counter()
  for i in range(epochs):
    for j, (data, label) in enumerate(train_dataloader):
      data = data.to(device)
      label = label.to(device)
      pred = model(data)
      loss = loss_fn(pred, label)

      op.zero_grad()
      loss.backward()
      op.step()

  end = time.perf_counter()

  print(f"batch {batch}: {(end - start)/epochs}")

batch 1: 9.06604509299999
batch 2: 4.585245211400002
batch 4: 2.3320711969000056
batch 8: 1.2051710414000127
batch 16: 0.6296191883999882
batch 32: 0.3846276741999873
batch 64: 0.20957397029999356
batch 128: 0.13744107749998874
